In [23]:
import tensorflow as tf
from transformers import BertTokenizer
import pandas as pd
import numpy as np
mdir = 'models/'
ddir = 'data/'

In [11]:
model = tf.keras.models.load_model(mdir+'sentiment_model_v1', compile = False)

In [13]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5, decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer = optimizer, loss = loss, metrics = [acc])

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [16]:
def prep_data(text):
    tokens = tokenizer(text, max_length = 512, truncation = True,
                      padding = 'max_length', add_special_tokens = True,
                      return_tensors = 'tf')
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask']}

In [18]:
probs = model.predict(prep_data('hello world'))[0]
probs

1/1 [==============================] - 1s 846ms/step


array([0.00521354, 0.03077669, 0.7011552 , 0.22871082, 0.0341437 ],
      dtype=float32)

In [21]:
pd.set_option('display.max_colwidth', None)

In [24]:
df = pd.read_csv(ddir+'test.tsv', sep = '\t')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
1,156062,8545,An intermittently pleasing but mostly routine effort
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [25]:
df = df.drop_duplicates(subset = ['SentenceId'], keep = 'first')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine effort .
15,156076,8546,"Kidman is really the only thing that 's worth watching in Birthday Girl , a film by the stage-trained Jez Butterworth -LRB- Mojo -RRB- that serves as yet another example of the sad decline of British comedies in the post-Full Monty world ."
93,156154,8547,Once you get into its rhythm ... the movie becomes a heady experience .
117,156178,8548,I kept wishing I was watching a documentary about the wartime Navajos and what they accomplished instead of all this specious Hollywood hoo-ha .
158,156219,8549,"Kinnear does n't aim for our sympathy , but rather delivers a performance of striking skill and depth ."


In [26]:
df['Sentiment'] = None

In [29]:
for i, row in df.iterrows():
    tokens = prep_data(row['Phrase'])
    probs = model.predict(tokens)
    pred = np.argmax(probs)
    df.at[i, 'Sentiment'] = pred

1/1 [==============================] - 1s 764ms/step


1/1 [==============================] - 1s 731ms/step


1/1 [==============================] - 1s 748ms/step


1/1 [==============================] - 1s 754ms/step


1/1 [==============================] - 1s 958ms/step


1/1 [==============================] - 1s 812ms/step


KeyboardInterrupt: 